In [1]:
import os
import shutil
import random
import numpy as np
import pandas as pd

import torch
from torch_geometric.data import Data

## Read grid info

In [2]:
# Create buses
num_buses = 118
buses = np.arange(1, num_buses+1)

# Read gen buses
file_path = '../../data/IEEE_Case118/zones/gen_bus.csv'
gen_buses = pd.read_csv(file_path, header=None, index_col=None)

# Read thermal buses
file_path = '../../data/IEEE_Case118/zones/thermal_bus.csv'
thermal_buses = pd.read_csv(file_path, header=None, index_col=None)

# Read wind gen buses
file_path = '../../data/IEEE_Case118/zones/wind_bus.csv'
wind_buses = pd.read_csv(file_path, header=None, index_col=None)

# Read load buses
file_path = '../../data/IEEE_Case118/zones/load_bus.csv'
load_buses = pd.read_csv(file_path, header=None, index_col=None)

In [3]:
# Boolean index of thermal buses
thermal_bidx = np.isin(buses, thermal_buses)

# Boolean index of wind buses
wind_bidx = np.isin(buses, wind_buses)

# Boolean index of load buses
load_bidx = np.isin(buses, load_buses)

In [4]:
# Get thermal gen features (To be determined)
Pmax = pd.read_csv('../../data/IEEE_Case118/gen_params/Pmax.csv',
                   header=None,
                   index_col=None)
Pmin = pd.read_csv('../../data/IEEE_Case118/gen_params/Pmin.csv',
                   header=None,
                   index_col=None)
ramp_30 = pd.read_csv('../../data/IEEE_Case118/gen_params/ramp_30.csv',
                      header=None,
                      index_col=None)
startup_cost = pd.read_csv('../../data/IEEE_Case118/gen_params/startup_cost.csv',
                            header=None,
                            index_col=None)
shutdown_cost = pd.read_csv('../../data/IEEE_Case118/gen_params/shutdown_cost.csv',
                            header=None,
                            index_col=None)
gencost_params = pd.read_csv('../../data/IEEE_Case118/gen_params/gencost_params.csv',
                                header=None,
                                index_col=None)
reserve_qty = pd.read_csv('../../data/IEEE_Case118/gen_params/reserve_qty.csv',
                            header=None,
                            index_col=None)
reserve_cost = pd.read_csv('../../data/IEEE_Case118/gen_params/reserve_cost.csv',
                            header=None,
                            index_col=None)

# Get the Boolean idx of thermals in gens
bidx = np.isin(gen_buses, thermal_buses).reshape((-1))

temp = np.concatenate([Pmax, Pmin, ramp_30, startup_cost, 
                  shutdown_cost, gencost_params], axis=1)

# Create thermal gens features
thermal_gen_features = np.concatenate([temp[bidx], reserve_qty, reserve_cost], axis=1)

In [5]:
# Time steps
nt = 12

# Read wind and load inputs into MATPOWER
num_samples = 1000

## DL ground truth

In [6]:
# Read MATPOWER UC solution
y_DL = []

# Add zeros to make the dimension as [num_buses, nt]
all_DL = np.zeros((num_buses, nt))

for i in range(num_samples):
    DL_path = f'../../data/IEEE_Case118/outputs/deployed_load/sample_{i+1}.csv'
    try:
        DL = pd.read_csv(DL_path, header=None, index_col=None).to_numpy()
    except FileNotFoundError:
        print(f'The file sample_{i+1} is not found')
        continue

    all_DL[load_bidx] = DL

    y_DL.append(all_DL.copy())

## Edge index and attributes

In [7]:
# Read branch info
file_path = '../../data/IEEE_Case118/branch_params/branch_params.csv'
branch = pd.read_csv(file_path, header=None, index_col=None).to_numpy()

# Read and assign PF_max
RATE_A = 5                  # Index of RATE_A
########################## These numbers are determined separately ##############################
file_path = f'../../data/IEEE_Case118/branch_params/PF_max_category1.csv'
PF_max_category1 = pd.read_csv(file_path, header=None, index_col=None).to_numpy().astype(bool).flatten()
file_path = f'../../data/IEEE_Case118/branch_params/PF_max_category2.csv'
PF_max_category2 = pd.read_csv(file_path, header=None, index_col=None).to_numpy().astype(bool).flatten()
file_path = f'../../data/IEEE_Case118/branch_params/PF_max_category3.csv'
PF_max_category3 = pd.read_csv(file_path, header=None, index_col=None).to_numpy().astype(bool).flatten()
PF_max1 = 200
PF_max2 = 200
PF_max3 = 200
branch[PF_max_category1, RATE_A] = PF_max1
branch[PF_max_category2, RATE_A] = PF_max2
branch[PF_max_category3, RATE_A] = PF_max3

# Get branch index and attr
edge_index = branch[:, :2] - 1
edge_attr = branch[:, 2:]

# Convert to standard format
edge_index = torch.tensor(edge_index.T, dtype=torch.long)
edge_attr = torch.from_numpy(edge_attr).float()

## SAGE dataset

In [8]:
# Empty list to store data
x_SAGE = []

# Get dimension of thermal, wind and load
num_thermal_features = thermal_gen_features.shape[1]

# Empty node feature matrix
node_features = np.zeros((buses.shape[0], num_thermal_features+nt*2))

# Assign thermal features
node_features[thermal_bidx, :num_thermal_features] = thermal_gen_features

# Assign wind and load features
for i in range(num_samples):
    # File path
    wind_path = f'../../data/IEEE_Case118/inputs/wind/sample_{i+1}.csv'
    load_path = f'../../data/IEEE_Case118/inputs/load/sample_{i+1}.csv'
    PG_path = f'./model_evaluation/SAGE/PG_pred_all/pred_{i+1}.csv'

    # Read wind and load
    try:
        wind = pd.read_csv(wind_path, header=None, index_col=None)
        load = pd.read_csv(load_path, header=None, index_col=None)
        PG = pd.read_csv(PG_path, header=None, index_col=None)
    except FileNotFoundError:
        print(f'The file sample_{i+1} is not found')
        continue

    # Assign node features
    node_features[wind_bidx, -2*nt:-nt] = wind
    node_features[thermal_bidx, -2*nt:-nt] = PG[thermal_bidx]
    node_features[load_bidx, -nt:] = load

    x_SAGE.append(node_features.copy())

In [9]:
# Check the statis of old dataset
dir = f'../../train_val_test_dataset/IEEE_Case118/DL-SAGE/processed'
if not os.path.exists(dir):
    print(f'There is no dataset found!')
else:
    shutil.rmtree(dir)
    print(f'The old dataset has been deleted!')

data_list = []

for i in range(len(x_SAGE)):
    X = torch.from_numpy(x_SAGE[i]).float()
    Y = torch.from_numpy(y_DL[i]).float()
    graph = Data(x=X, y=Y, edge_index=edge_index, edge_attr=edge_attr)
    data_list.append(graph)

from MyDataset import MyDataset
# Not that if there is already saved dataset, this cell won't work
# Save train, val and test data
root = '../../train_val_test_dataset/IEEE_Case118/DL-SAGE'
MyDataset(root=root, data_list=data_list)

The old dataset has been deleted!


Processing...
Done!


MyDataset(1000)

## ANN dataset

In [10]:
# Empty list to store data
x_ANN = []

# Get dimension of thermal, wind and load
num_thermal_features = thermal_gen_features.shape[1]

# Empty node feature matrix
node_features = np.zeros((buses.shape[0], num_thermal_features+nt*2))

# Assign thermal features
node_features[thermal_bidx, :num_thermal_features] = thermal_gen_features

# Assign wind and load features
for i in range(num_samples):
    # File path
    wind_path = f'../../data/IEEE_Case118/inputs/wind/sample_{i+1}.csv'
    load_path = f'../../data/IEEE_Case118/inputs/load/sample_{i+1}.csv'
    PG_path = f'./model_evaluation/ANN/PG_pred_all/pred_{i+1}.csv'

    # Read wind and load
    try:
        wind = pd.read_csv(wind_path, header=None, index_col=None)
        load = pd.read_csv(load_path, header=None, index_col=None)
        PG = pd.read_csv(PG_path, header=None, index_col=None)
    except FileNotFoundError:
        print(f'The file sample_{i+1} is not found')
        continue

    # Assign node features
    node_features[wind_bidx, -2*nt:-nt] = wind
    node_features[thermal_bidx, -2*nt:-nt] = PG[thermal_bidx]
    node_features[load_bidx, -nt:] = load

    x_ANN.append(node_features.copy())

In [11]:
# Check the statis of old dataset
dir = f'../../train_val_test_dataset/IEEE_Case118/DL-ANN/processed'
if not os.path.exists(dir):
    print(f'There is no dataset found!')
else:
    shutil.rmtree(dir)
    print(f'The old dataset has been deleted!')

data_list = []

for i in range(len(x_ANN)):
    X = torch.from_numpy(x_ANN[i]).float()
    Y = torch.from_numpy(y_DL[i]).float()
    graph = Data(x=X, y=Y, edge_index=edge_index, edge_attr=edge_attr)
    data_list.append(graph)

from MyDataset import MyDataset
# Not that if there is already saved dataset, this cell won't work
# Save train, val and test data
root = '../../train_val_test_dataset/IEEE_Case118/DL-ANN'
MyDataset(root=root, data_list=data_list)

The old dataset has been deleted!


Processing...
Done!


MyDataset(1000)

## GCN dataset

In [12]:
# Empty list to store data
x_GCN = []

# Get dimension of thermal, wind and load
num_thermal_features = thermal_gen_features.shape[1]

# Empty node feature matrix
node_features = np.zeros((buses.shape[0], num_thermal_features+nt*2))

# Assign thermal features
node_features[thermal_bidx, :num_thermal_features] = thermal_gen_features

# Assign wind and load features
for i in range(num_samples):
    # File path
    wind_path = f'../../data/IEEE_Case118/inputs/wind/sample_{i+1}.csv'
    load_path = f'../../data/IEEE_Case118/inputs/load/sample_{i+1}.csv'
    PG_path = f'./model_evaluation/GCN/PG_pred_all/pred_{i+1}.csv'

    # Read wind and load
    try:
        wind = pd.read_csv(wind_path, header=None, index_col=None)
        load = pd.read_csv(load_path, header=None, index_col=None)
        PG = pd.read_csv(PG_path, header=None, index_col=None)
    except FileNotFoundError:
        print(f'The file sample_{i+1} is not found')
        continue

    # Assign node features
    node_features[wind_bidx, -2*nt:-nt] = wind
    node_features[thermal_bidx, -2*nt:-nt] = PG[thermal_bidx]
    node_features[load_bidx, -nt:] = load

    x_GCN.append(node_features.copy())

In [13]:
# Check the statis of old dataset
dir = f'../../train_val_test_dataset/IEEE_Case118/DL-GCN/processed'
if not os.path.exists(dir):
    print(f'There is no dataset found!')
else:
    shutil.rmtree(dir)
    print(f'The old dataset has been deleted!')

data_list = []

for i in range(len(x_GCN)):
    X = torch.from_numpy(x_GCN[i]).float()
    Y = torch.from_numpy(y_DL[i]).float()
    graph = Data(x=X, y=Y, edge_index=edge_index, edge_attr=edge_attr)
    data_list.append(graph)

from MyDataset import MyDataset
# Not that if there is already saved dataset, this cell won't work
# Save train, val and test data
root = '../../train_val_test_dataset/IEEE_Case118/DL-GCN'
MyDataset(root=root, data_list=data_list)

The old dataset has been deleted!


Processing...
Done!


MyDataset(1000)